In [1]:
# make sure jupyter path is correct for loading local moudules
import sys
# path to steric_simulator module relative to notebook
sys.path.append("../../../")
import copy
from steric_free_simulator import ReactionNetwork, VectorizedRxnNet, VecSim, Optimizer, EquilibriumSolver

EnergyExplorer Module is not available. Check Rosetta installation. <ipykernel.iostream.OutStream object at 0x7f71c1c18cc0>


In [2]:
base_input = '../../input_files/trimer.pwr'
rn = ReactionNetwork(base_input, one_step=True)
rn.resolve_tree()

['A']
['B']
['C']
-----
{'A'}
{'A'}
set()
-----
{'A'}
{'B'}
{'A'}
False
Allowed edges: 
('A', 'B')
Connected Nodes:  ['A', 'B']
Connected Edges:  [('A', 'B')]
Allowed edges: 
('A', 'C')
Allowed edges: 
('B', 'C')
New node added--1
['A', 'B']
{('A', 'B'): [None, None, 1, tensor([-50000.], dtype=torch.float64)], ('A', 'C'): [None, None, 1, tensor([-50000.], dtype=torch.float64)], ('B', 'C'): [None, None, 1, tensor([-50000.], dtype=torch.float64)]}
New node added--2
[0, 1, 2, 3]
-----
{'A'}
{'C'}
{'A'}
False
Allowed edges: 
('A', 'B')
Allowed edges: 
('A', 'C')
Connected Nodes:  ['A', 'C']
Connected Edges:  [('A', 'C')]
Allowed edges: 
('B', 'C')
New node added--1
['A', 'C']
{('A', 'B'): [None, None, 1, tensor([-50000.], dtype=torch.float64)], ('A', 'C'): [None, None, 1, tensor([-50000.], dtype=torch.float64)], ('B', 'C'): [None, None, 1, tensor([-50000.], dtype=torch.float64)]}
New node added--2
[0, 1, 2, 3, 4]
Allowed edges: 
('A', 'B')
Allowed edges: 
('A', 'C')
Allowed edges: 
('B', '

In [3]:
uid_dict = {}
sys.path.append("../../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_dict[(n,k)] = uid

print(uid_dict)

{(0, 3): 0, (0, 4): 1, (0, 6): 5, (1, 3): 0, (1, 5): 2, (1, 6): 3, (2, 4): 1, (2, 5): 2, (2, 6): 4, (3, 6): 4, (4, 6): 3, (5, 6): 5}


In [4]:
sys.path.append("../../")
import numpy as np
from reaction_network import gtostr
from torch import DoubleTensor as Tensor
import torch

node_map = {}
for node in rn.network.nodes():
    node_map[gtostr(rn.network.nodes[node]['struct'])] = node
def get_max_edge(vec_rn,n,node_map):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        if len(edges)==0:
            return(False)
        kon_max = -1
        next_node = -1
        
        kon_sum = 0
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']
            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)
def get_node_flux(vec_rn,n):
    total_flux_outedges = 0
    total_flux_inedges = 0
    #Go over all the out edges
    edges_out = rn.network.out_edges(n)
    if len(edges_out)>0:

        for edge in edges_out:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]

            #Calculate k_off also
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Getting conc. of reactants and products
            #Get product
            prod = gtostr(rn.network.nodes[edge[1]]['struct']) 
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))

            #Net flux from this edge = Generation - consumption
            edge_flux = koff*vec_rn.copies_vec[edge[1]] - temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])
            #edge_flux = koff*vec_rn.copies_vec[edge[1]] 

#             print("Reaction: ", gtostr(rn.network.nodes[edge[0]]['struct']), "+",react," -> ",prod)
#             print("Net flux: ",edge_flux)
#             print("kon : ",temp_kon)
#             print("koff: ",koff)
#             print("Reaction data OUTWARD: ")
#             print(data)

            total_flux_outedges+=edge_flux
    
    #Now go over all the in edges
    edges_in = rn.network.in_edges(n)
    react_list = []
    if len(edges_in) > 0:
        for edge in edges_in:
            if edge[0] in react_list:
                continue
            data = rn.network.get_edge_data(edge[0],edge[1])
            uid = data['uid']


            #Get generation rates; which would be kon
            temp_kon = vec_rn.kon[uid]

            #Get consumption rates; which is k_off
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Get conc. of reactants and products
            prod = gtostr(rn.network.nodes[edge[1]]['struct'])
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))
            react_list.append(node_map[react])
            #Net flux from this edge = Generation - consumption
            edge_flux_in = temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])- koff*vec_rn.copies_vec[edge[1]]
            #edge_flux_in = koff*vec_rn.copies_vec[edge[1]]
            


#             print("Reaction: ", prod ," -> ",gtostr(rn.network.nodes[edge[0]]['struct']), "+",react)
#             print("Net flux: ",edge_flux_in)
#             print("kon : ",temp_kon)
#             print("koff: ",koff)
#             print("Raction data INWARD: ")
#             print(data)

            total_flux_inedges+=edge_flux_in
    net_node_flux = total_flux_outedges + total_flux_inedges
    
    return(net_node_flux)

def do_flux_analysis(vec_rn,node_map):
    pathway = []
    kon_sumarray = []
    total_con_rate = {}
    net_flux = {}
    for n in rn.network.nodes():

        n_str = gtostr(rn.network.nodes[n]['struct']) 

        paths = [n_str]
        kon_sum = 0
        temp_node = n
        max_edge = True
        consumption_rate = 0
        if n < len(rn.network.nodes()):#num_monomers:
    #         print("Current node: ")
    #         print(n_str)
            while max_edge:
                max_edge = get_max_edge(vec_rn,temp_node,node_map)
                if max_edge:
                    total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                    temp_node = max_edge[1]
                    kon_sum += max_edge[0].item()


    #                 print("Next node: ")
    #                 print(temp_node)

                    paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
                else:
                    break
            pathway.append(paths)
            kon_sumarray.append(kon_sum)
            paths=[]
            
        net_flux[gtostr(rn.network.nodes[n]['struct'])] = get_node_flux(vec_rn,n)

    #print(pathway)
    #print(kon_sumarray)
    
    return(total_con_rate,net_flux)

In [ ]:
## Changing Initial Conditions
import networkx as nx
#Changin k_on
kon_list = [10.0,100.0]# uM-1 s-1

final_yield = {kon : [] for kon in kon_list}
optimized_rates = {kon : [] for kon in kon_list}
observables = {kon : dict() for kon in kon_list} 
flux_data = {kon : [] for kon in kon_list}
cons_data = {kon : [] for kon in kon_list}
final_copies = {kon : [] for kon in kon_list}

for new_kon in kon_list:

    nx.set_edge_attributes(rn.network,new_kon,'k_on')
    
    
    vec_rn = VectorizedRxnNet(rn, dev='cpu')
    vec_rn.reset(reset_params=True)
    
    ### Start Optimization
    print("Running Optimization with the following on rates: ")
    print(vec_rn.kon)
    optim = Optimizer(reaction_network=vec_rn,
                      sim_runtime=5,
                      optim_iterations=10000,
                      learning_rate=new_kon/1000,
                      device='cpu')
    optim.rn.update_reaction_net(rn)
    optim.optimize()
    
    final_yield[new_kon] = optim.yield_per_iter
    optimized_rates[new_kon] = vec_rn.kon
    
    ###Run simulation with optimized parameters
    print("Running simulation with optimized parameters")
    optim_rn = optim.rn
    for i, runtime in enumerate([1, 50, 100,1000]):
        optim_rn.reset()
        sim = VecSim(optim_rn, runtime, device='cpu')
        y = sim.simulate()
        
        observables[new_kon][runtime] = (sim.steps.copy(),sim.observables.copy())
        print("Length of observables: ", len(sim.observables[0][1]))
    
    ##Calculate the net flux in across each node/species
    print("Doing flux analysis")
    cons_rate,flux = do_flux_analysis(vec_rn,node_map)
    
    flux_data[new_kon].append(flux)
    cons_data[new_kon].append(cons_rate)
    final_copies[new_kon].append(vec_rn.copies_vec)
    
        
    

Running Optimization with the following on rates: 
Parameter containing:
tensor([10., 10., 10., 10., 10., 10.], dtype=torch.float64, requires_grad=True)
Using CPU
Reaction Parameters before optimization: 
[Parameter containing:
tensor([10., 10., 10., 10., 10., 10.], dtype=torch.float64, requires_grad=True)]
Using CPU
Current time:  tensor(4.9945, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0035, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 0 was 50.3%
current params: tensor([ 9.9900,  9.9900,  9.9900, 10.0100, 10.0100, 10.0100],
       dtype=torch.float64)
Using CPU
Current time:  tensor(4.9937, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0026, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 1 was 50.4%
current params: tensor([ 9.9800,  9.9800,  9.9800, 10.0200, 10.0200, 10.0200],

Current time:  tensor(4.9950, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0040, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 22 was 51.2%
current params: tensor([ 9.7697,  9.7697,  9.7695, 10.2295, 10.2295, 10.2299],
       dtype=torch.float64)
Using CPU
Current time:  tensor(4.9944, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0033, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 23 was 51.3%
current params: tensor([ 9.7597,  9.7597,  9.7595, 10.2395, 10.2395, 10.2398],
       dtype=torch.float64)
Using CPU
Current time:  tensor(4.9937, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0026, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 24 was 51.3%
current params: tensor([ 9.7496,  

Current time:  tensor(4.9994, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0083, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 45 was 52.2%
current params: tensor([ 9.5380,  9.5380,  9.5376, 10.4570, 10.4570, 10.4578],
       dtype=torch.float64)
Using CPU
Current time:  tensor(4.9989, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0078, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 46 was 52.2%
current params: tensor([ 9.5279,  9.5279,  9.5274, 10.4669, 10.4669, 10.4677],
       dtype=torch.float64)
Using CPU
Current time:  tensor(4.9985, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0074, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 47 was 52.2%
current params: tensor([ 9.5178,  

Current time:  tensor(4.9999, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0088, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 68 was 53.1%
current params: tensor([ 9.3044,  9.3044,  9.3035, 10.6817, 10.6817, 10.6831],
       dtype=torch.float64)
Using CPU
Current time:  tensor(4.9997, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0086, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 69 was 53.1%
current params: tensor([ 9.2942,  9.2942,  9.2932, 10.6914, 10.6914, 10.6929],
       dtype=torch.float64)
Using CPU
Current time:  tensor(4.9995, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0083, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 70 was 53.2%
current params: tensor([ 9.2840,  

Current time:  tensor(4.9968, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0057, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 91 was 54.0%
current params: tensor([ 9.0685,  9.0685,  9.0670, 10.9034, 10.9034, 10.9058],
       dtype=torch.float64)
Using CPU
Current time:  tensor(4.9968, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0057, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 92 was 54.1%
current params: tensor([ 9.0582,  9.0582,  9.0566, 10.9130, 10.9130, 10.9154],
       dtype=torch.float64)
Using CPU
Current time:  tensor(4.9968, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0057, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 93 was 54.1%
current params: tensor([ 9.0479,  

...

Current time:  tensor(4.9978, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0067, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 137 was 55.9%
current params: tensor([ 8.5898,  8.5898,  8.5864, 11.3383, 11.3383, 11.3432],
       dtype=torch.float64)
Using CPU
Current time:  tensor(4.9983, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0072, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 138 was 55.9%
current params: tensor([ 8.5793,  8.5793,  8.5759, 11.3476, 11.3476, 11.3526],
       dtype=torch.float64)
Using CPU
Current time:  tensor(4.9988, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0077, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 139 was 56.0%
current params: tensor([ 8.5688

Current time:  tensor(4.9932, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0022, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 160 was 56.8%
current params: tensor([ 8.3469,  8.3469,  8.3422, 11.5515, 11.5515, 11.5581],
       dtype=torch.float64)
Using CPU
Current time:  tensor(4.9940, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0029, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 161 was 56.9%
current params: tensor([ 8.3363,  8.3363,  8.3315, 11.5607, 11.5607, 11.5674],
       dtype=torch.float64)
Using CPU
Current time:  tensor(4.9947, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0036, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 162 was 56.9%
current params: tensor([ 8.3256

Current time:  tensor(4.9944, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0033, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 183 was 57.8%
current params: tensor([ 8.1014,  8.1014,  8.0953, 11.7620, 11.7620, 11.7705],
       dtype=torch.float64)
Using CPU
Current time:  tensor(4.9953, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0043, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 184 was 57.8%
current params: tensor([ 8.0906,  8.0906,  8.0845, 11.7711, 11.7711, 11.7797],
       dtype=torch.float64)
Using CPU
Current time:  tensor(4.9963, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0053, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 185 was 57.8%
current params: tensor([ 8.0799

Current time:  tensor(4.9925, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0015, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 206 was 58.7%
current params: tensor([ 7.8533,  7.8533,  7.8455, 11.9699, 11.9699, 11.9805],
       dtype=torch.float64)
Using CPU
Current time:  tensor(4.9937, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0027, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 207 was 58.7%
current params: tensor([ 7.8424,  7.8424,  7.8346, 11.9789, 11.9789, 11.9895],
       dtype=torch.float64)
Using CPU
Current time:  tensor(4.9950, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0040, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 208 was 58.8%
current params: tensor([ 7.8316

Current time:  tensor(4.9967, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0058, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 229 was 59.7%
current params: tensor([ 7.6024,  7.6024,  7.5927, 12.1751, 12.1751, 12.1879],
       dtype=torch.float64)
Using CPU
Current time:  tensor(4.9983, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0073, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 230 was 59.7%
current params: tensor([ 7.5915,  7.5915,  7.5817, 12.1839, 12.1839, 12.1969],
       dtype=torch.float64)
Using CPU
Current time:  tensor(4.9998, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(5.0089, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 231 was 59.7%
current params: tensor([ 7.5805

In [ ]:
##Plotting yield for each optimized rates
from matplotlib import pyplot as plt
%matplotlib inline
fig, ax = plt.subplots(1, len(kon_list))
i= 0
for kon,val in final_yield.items():
    ax[i].plot(val)
    ax[i].set_title("Initial kon = " + str(kon))
    i+=1

txt = fig.suptitle("Yield after optimization")
fig.set_size_inches(24, 6)
fig.tight_layout()
plt.show()

In [ ]:
times = 4
fig,ax = plt.subplots(len(kon_list),times)
i=0
nodes_list = ['A','B','C','AB','BC','AC','ABC']
for kon,runtimes in observables.items():
    for time,val in runtimes.items():
        #print(time)
        for key in val[1].keys():
            if val[1][key][0] in nodes_list:
                row = int(i/times)
                col = i%times
                #print(val[1][key][1][:5])
                #print(len(val[1][key][1]))
                ax[row,col].plot(val[0],val[1][key][1],label=val[1][key][0])
        ax[row,col].set_title("runtime: " + str(time) + " seconds")
        i+=1
lgnd = plt.legend(loc='best')
fig.set_size_inches(18, 24)
fig.tight_layout()
plt.show()

In [ ]:
print(int(4/4))

In [ ]:
for kon,flux in flux_data.items():
    print("-----------------------")
    print("Starting on rate: ", kon)
    print("Final copies: ", final_copies[kon])
    print("Optimzied parameters: ", optimized_rates[kon])
    
    print("Flux: ")
    for k,v in sorted(flux[0].items(),key=lambda x : x[1]):
        print(k," : ", v)